<a href="https://colab.research.google.com/github/artemislong/dumpster/blob/main/Copy_of_Quiz_10_Frequent_Itemsets_Mining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# **Frequent Itemsets Mining with PySpark in Colab**

To run spark in Colab, we need to first install all the dependencies in Colab environment i.e. Apache Spark 2.3.2 with hadoop 2.7, Java 8 and Findspark to locate the spark in the system.

Follow the steps to install the dependencies:

In [42]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [43]:
!wget -qN https://archive.apache.org/dist/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz

In [44]:
!pip install -q findspark

Set the location of Java and Spark by running the following code:

In [45]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "spark-3.2.1-bin-hadoop3.2"

Install PySpark and run a local spark session to test the installation:

In [46]:
!pip install pyspark

In [6]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = spark.sparkContext

<div style="page-break-before: always;"></div>

## Q1. Support and Confidence [9 points] ✅

Suppose there are 100 items, numbered 1 to 100, and also 100 baskets,also numbered 1 to 100. **Item $i$ is in basket $b$ if and only if $i$ divides $b$ with no remainder.** Thus, item 1 is in all the baskets, item 2 is in all fifty of the even-numbered baskets, and so on. Basket 12 consists of items {1, 2, 3, 4, 6, 12}, since these are all the integers that divide 12. Answer the following questions:

1. If the support threshold is 5, which items are frequent?

2. For the above data, what is the confidence of the association rule {5, 7} → 2?
2. the confidence of the association rule {2, 3, 4} → 5?

(Exercises 6.1.1 and 6.1.5 from the Stanford book)

In [47]:
import numpy as np
items = np.array(range(1, 101))
baskets = {i: [] for i in range(1, 101)}
appearance = {i: [] for i in range(1, 101)} # track where item appears

for i in items:
  for j in range(i, 101):
    if j % i == 0:
      baskets[j].append(i)
      appearance[i].append(j)
# print(appearance)

s = 5 # support threshold
frequent_items = []
# finding frequent items
for i in items:
  count = len(appearance[i])
  if count >= s: frequent_items.append(i)
print("Items with frequency above s=5: ",frequent_items)

# 2: confidence of {5,7} -> 2
print("Part 2: confidence of {5,7} -> 2")
def find_itemset(item, baskets, indexes):
  listi = []
  for i in indexes:
    # print(baskets[i])
    if item in baskets[i]:
      listi.append(i)
  return listi # returns indexes of sets that contains itemset

list57 = find_itemset(7, baskets, appearance[5])
print("found 5,7 in:",list57)
# check for 2
list572 = find_itemset(2, baskets, list57)
print("found 5,7,2 in:",list572)
# confidence
confidence572 = len(list572) / len(list57)
print("confidence of the association rule {5, 7} → 2 is: ", confidence572)

# 3: confidence of the association rule {2, 3, 4} → 5
print("Part 3: confidence of the association rule {2, 3, 4} → 5")
list23 = find_itemset(3, baskets, appearance[2])
print("found 2,3 in:",list23)
# check for 4
list234 = find_itemset(4, baskets, list23)
print("found 2,3,4 in:",list234)
# check for 5
list2345 = find_itemset(5, baskets, list234)
print("found 2,3,4,5 in:",list2345)
# confidence
confidence2345 = len(list2345) / len(list234)
print("confidence of the association rule {2,3,4} → 5 is: ", confidence2345)

Items with frequency above s=5:  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
Part 2: confidence of {5,7} -> 2
found 5,7 in: [35, 70]
found 5,7,2 in: [70]
confidence of the association rule {5, 7} → 2 is:  0.5
Part 3: confidence of the association rule {2, 3, 4} → 5
found 2,3 in: [6, 12, 18, 24, 30, 36, 42, 48, 54, 60, 66, 72, 78, 84, 90, 96]
found 2,3,4 in: [12, 24, 36, 48, 60, 72, 84, 96]
found 2,3,4,5 in: [60]
confidence of the association rule {2,3,4} → 5 is:  0.125


<div style="page-break-before: always;"></div>

### Tiny Data Setup for Frequent Itemset Mining ✅

Let's create a spark DataFrame to confirm that we can run PySpark, and preload that DataFrame with test baskets.

| Transaction ID | Stock Items |
|:--------------:|:---------------- |
| 100 | milk, coke, beer |
| 200 | milk, pepsi, juice |
| 300 | milk, beer |
| 400 | coke, juice |
| 500 | milk, pepsi, beer |
| 600 | milk, coke, beer, juice |
| 700 | coke, beer, juice |
| 800 | beer, coke |

Each DataFrame row is `<Transaction ID, [Stock Items]>`

In [48]:
import numpy as np

In [49]:
# No coding required for this cell, just run it
m,c,b,p,j = 12,3,2,15,9
basket_df = spark.createDataFrame([
    (100, [m,c,b]),
    (200, [m,p,j]),
    (300, [m,b]),
    (400, [c,j]),
    (500, [m,p,b]),
    (600, [m,c,b,j]),
    (700, [c,b,j]),
    (800, [b,c])
], ["id", "items"])
basket_df.show()
stockIDs = {b: 'Beer', c: 'Coke', m: 'Milk', j: 'Juice', p: 'Pepsi'}

+---+-------------+
| id|        items|
+---+-------------+
|100|   [12, 3, 2]|
|200|  [12, 15, 9]|
|300|      [12, 2]|
|400|       [3, 9]|
|500|  [12, 15, 2]|
|600|[12, 3, 2, 9]|
|700|    [3, 2, 9]|
|800|       [2, 3]|
+---+-------------+



<div style="page-break-before: always;"></div>


## FP-Growth Algorithm on the Tiny Dataset ✅

Ready to run FP-Growth? References:

* PySpark [introduction for FP-Growth](https://spark.apache.org/docs/latest/ml-frequent-pattern-mining.html#fp-growth).
* [PySpark Dataframes](https://sparkbyexamples.com/pyspark/convert-pandas-to-pyspark-dataframe/)

Let's first run FP-Growth example from the documentation

In [50]:
# No coding required for this cell, just run it
from pyspark.ml.fpm import FPGrowth

fpGrowth = FPGrowth(itemsCol="items", minSupport=0.5, minConfidence=0.6)
model = fpGrowth.fit(basket_df)

# Skipping display of frequent itemsets.
# model.freqItemsets.show()

# Display generated association rules.
model.associationRules.show()

+----------+----------+------------------+------------------+-------+
|antecedent|consequent|        confidence|              lift|support|
+----------+----------+------------------+------------------+-------+
|       [3]|       [2]|               0.8|1.0666666666666667|    0.5|
|      [12]|       [2]|               0.8|1.0666666666666667|    0.5|
|       [2]|       [3]|0.6666666666666666|1.0666666666666667|    0.5|
|       [2]|      [12]|0.6666666666666666|1.0666666666666667|    0.5|
+----------+----------+------------------+------------------+-------+



## Q2. Interpreting association rules [6 points] ✅

The above table, has columns antecedent, consequent, confidence, lift and support.

**1. Explain the first row, `[3] [2] 0.8 1.0666666666666667 0.5` in plain English.**


The association rule {3} -> 2 has confidende score of 0.8, which is the probability of itemset having 2 given that it contains 3. Lift is 1.067, which means 2 is 1.067 times more likely to be in the set if 3 is present.

0.5 is the support of this rule. the rule applies to 50% of itemsets.

**2. The first and the third rows have the antecedent and consequent switched, but different confidence values. (Same with second and fourth rows). How do you explain those results?**

If the probability of an itemset containing A given it contains B is lower than the reverse (probability of itemset containing B given it contains A) indicates that there are more itemsets that contain A compared to itemsets with B (since the number of itemsets with both A and B is the same).

**3. What does support = 0.5 for all the rows mean?**
That means the association rule applies to half of itemsets. I.e. half of sets contain both 2 and 3 if association (3,2) has support of 0.5

<div style="page-break-before: always;"></div>


## Q3. Association Rules with new `minSupport` and `minConfidence` values [2 pts] ✅

Modify the support threshold to be 0.375 and minimum confidence to be 0.75 to make the parameters consistent with the settings in the textbook.

In [53]:
# No coding required for this cell, just run it
from pyspark.ml.fpm import FPGrowth

fpGrowth = FPGrowth(itemsCol="items", minSupport=0.375, minConfidence=0.75)
model = fpGrowth.fit(basket_df)

# Display frequent itemsets.
model.freqItemsets.show()

# Display generated association rules.
model.associationRules.show()

+-------+----+
|  items|freq|
+-------+----+
|    [3]|   5|
| [3, 2]|   4|
|    [2]|   6|
|   [12]|   5|
|[12, 2]|   4|
|    [9]|   4|
| [9, 3]|   3|
+-------+----+

+----------+----------+----------+------------------+-------+
|antecedent|consequent|confidence|              lift|support|
+----------+----------+----------+------------------+-------+
|       [3]|       [2]|       0.8|1.0666666666666667|    0.5|
|      [12]|       [2]|       0.8|1.0666666666666667|    0.5|
|       [9]|       [3]|      0.75|               1.2|  0.375|
+----------+----------+----------+------------------+-------+



## Q4. Low Support and High Lift [5 pts] ✅

**The third row of the result shows a low support (0.375) and a high lift (1.2). What does this line tell us?**

Low support (0.375) indicates that this itemset {9,3} is not common in the corpus (it appears in 3 out 8 sets, hence 3/8=0.375 support). High lift means {3} is more likely to be in a set if {9} is present, in this case {3} is 1.2 times to be present in sets that contain {9}.

The relationship is visible to manual inspection: both {3} and {9} appeared in 3 sets, while frequency of {3} and {9} is 5 and 4 respectively. So 3/4 sets that contain {9} also contain {3}, which is indeed indicated by high lift value.



<div style="page-break-before: always;"></div>


## Q5. An Online Retail Dataset [0 pts, nothing to do] ✅

The main part of this exercise involves processing a sampled dataset from a UK-based online retailer. We'll be working with a 8050 record subset.

* Read in the data from the dataset `online_retail_II.csv`. For your convenience, code to throw away bad records using `dropna()` is included.
* There are a couple of wrinkles to keep in mind in case you are curious, though you may not really need them.
    * An invoice represents a shopping cart and it can contain multiple items.
    * Some invoice numbers start with a "C." Invoice number C123456 is to be interpreted as a return of items in invoice 123456. The `inum` column represents the Invoice number as well as the credit (return). In other words, Invoice numbers `123456` and `C123456` would have `inum` == 123456.

In [66]:
# No coding required for this cell, just run it

import pandas as pd

df_orig = pd.read_csv('https://andrewwinslow.com/online_retail_II.csv')
df_orig.dropna(inplace=True)
df_orig

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,12/1/09 7:45,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,12/1/09 7:45,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,12/1/09 7:45,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,12/1/09 7:45,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,12/1/09 7:45,1.25,13085.0,United Kingdom
...,...,...,...,...,...,...,...,...
1067366,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12/9/11 12:50,2.10,12680.0,France
1067367,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12/9/11 12:50,4.15,12680.0,France
1067368,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12/9/11 12:50,4.15,12680.0,France
1067369,581587,22138,BAKING SET 9 PIECE RETROSPOT,3,12/9/11 12:50,4.95,12680.0,France


<div style="page-break-before: always;"></div>


## Q6. Data Scrubbing [5 pts] ✅

Remove the rows we should filter away. They aren't necessarily visible in the summary view but we know they exist.

* StockCode `POST`,
* StockCode `M`.


In [78]:
# Drop rows where 'StockCode' contains "POST"
df_clean = df_orig[~df_orig['StockCode'].str.contains('POST', case=False)] # ~ = not, keeping rows that do not contain POST
df_clean = df_clean[~df_clean['StockCode'].str.contains('M', case=False)]
# also dropping return invoices that start with "C"
df_clean = df_clean[~df_clean['Invoice'].str.contains('C', case=False)]

print("Data Scrubbed, remaining no. of rows: ", len(df_clean), ", previous no. of rows: ", len(df_orig), ", removed: ", len(df_orig) - len(df_clean), "rows")

Data Scrubbed, remaining no. of rows:  802327 , previous no. of rows:  824364 , removed:  22037 rows


<div style="page-break-before: always;"></div>


## Q7. Connecting Online Retail Data to FP-Growth [10 pts] ✅

Adapt the DataFrame to look like `df_basket` above.
* `df_orig` is a Pandas DataFrame whereas `df_basket`-equivalent will have to be Spark DataFrames.
* `Invoice` and `StockCode` are strings but FP-Growth needs inputs to be integers. You'd need to map strings to integers before feeding them to FP-Growth and convert the resulting antecedents and consequents back.

### Establish the mapping between Invoice IDs, StockCodes and unique integers.

In [79]:
# converting Invoice and StockCode to integers
def convert_letters_to_numbers(s): # replace letters with numbers
    result = ''
    # remove whitespace
    s = s.replace(" ", "")
    for char in s:
        if char.isalpha(): # if a letter, replace with alphabetic index
            result += str(ord(char.upper()) - ord('A') + 1)
        else:
            result += char
    return result
df_clean['Invoice'] = pd.to_numeric(df_clean['Invoice'].apply(convert_letters_to_numbers))
df_clean['StockCode'] = pd.to_numeric(df_clean['StockCode'].apply(convert_letters_to_numbers))

# reducing the dataset by invoice number
df_reduced = df_clean.groupby('Invoice')['StockCode'].agg(list).reset_index()
df_reduced


,Invoice,StockCode
0,489434,"[85048, 7932316, 7932323, 22041, 21232, 22064,..."
1,489435,"[22350, 22349, 22195, 22353]"
2,489436,"[481733, 21755, 21754, 84879, 22119, 22142, 22..."
3,489437,"[22143, 22145, 22130, 21364, 21360, 21351, 213..."
4,489438,"[21329, 21252, 21100, 21033, 20711, 21410, 214..."
...,...,...
36660,581583,"[20725, 85038]"
36661,581584,"[20832, 85038]"
36662,581585,"[22481, 22915, 22178, 22460, 84832, 23084, 848..."
36663,581586,"[22061, 23275, 21217, 20685]"


In [98]:
# Converting to Spark Dataframe
# First we convert to list to tuples
df_list = np.array(list(df_reduced.itertuples(index=False, name=None)))
# getting rid of redundant items
for i in range(np.shape(df_list)[0]):
  before = df_list[i][1]
  df_list[i][1] = list(set(before))
df_list = df_list.tolist()

# DataFrame
df_retail = spark.createDataFrame(df_list, ["id", "items"])
df_retail.show()

<ipython-input-98-6223a5bde7f6>:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  df_list = np.array(list(df_reduced.itertuples(index=False, name=None)))


[[489434, [7932323, 21871, 21232, 22064, 21523, 85048, 22041, 7932316]], [489435, [22353, 22195, 22349, 22350]], [489436, [845966, 84879, 82582, 22295, 22296, 22194, 21181, 481733, 21333, 350042, 22107, 8459612, 22109, 22111, 22119, 21754, 21755, 21756, 22142]], [489437, [22272, 22145, 22274, 845072, 10002, 21912, 22143, 35400, 20695, 20703, 22111, 22112, 21987, 21989, 21351, 21352, 20971, 21360, 22130, 21364, 37370, 8497019, 22271]], [489438, [840321, 21410, 21411, 21252, 840322, 20711, 845191, 21033, 845192, 21100, 851831, 21329, 851832, 840311, 840312, 851321, 851323]], [489439, [85216, 852322, 21731, 22333, 20749, 22352, 22065, 1616914, 21491, 1616116, 21493, 22130, 22064, 84691, 22138, 22139, 850141, 850142]], [489440, [22349, 22350]], [489441, [22321, 22138, 22111, 840295]], [489442, [22272, 21888, 22024, 22025, 22026, 22029, 22031, 8425110, 22296, 21916, 21790, 851231, 21428, 21430, 85178, 21955, 22091, 21582, 21586, 21590, 22111, 848995, 22271]], [489443, [21035, 21041, 20754, 

<div style="page-break-before: always;"></div>


## Q8. Fine-tuning FP-Growth runs [5 pts] ✅

* Set `minConfidence` = 0.75.
* Set `minSupport` such that the total number of association rules is between 10 and 20. (If `minSupport` is small, the number of association rules will increase. As it increases, the number of association rules will decrease.).



In [105]:
fpGrowth = FPGrowth(itemsCol="items", minSupport=0.01, minConfidence=0.75)
model_retail = fpGrowth.fit(df_retail)

# Display frequent itemsets.
model_retail.freqItemsets.show()

# Display generated association rules.
model_retail.associationRules.show()

+--------+----+
|   items|freq|
+--------+----+
| [21328]| 374|
| [22029]| 861|
| [22084]| 704|
|[475914]|1034|
|[850493]| 459|
| [20749]| 726|
| [21243]| 395|
| [21429]| 764|
| [22151]| 645|
| [22631]| 469|
|[475562]| 408|
| [21172]| 804|
| [22089]| 422|
|[840295]| 956|
| [22654]| 440|
| [21340]| 383|
| [22952]| 904|
| [20982]| 487|
| [20711]| 671|
| [22746]| 460|
+--------+----+
only showing top 20 rows

+---------------+----------+------------------+------------------+--------------------+
|     antecedent|consequent|        confidence|              lift|             support|
+---------------+----------+------------------+------------------+--------------------+
|        [22748]|   [22745]|0.7662337662337663| 50.52870690460619|0.012873312423291967|
|        [22745]|   [22748]|0.8489208633093526| 50.52870690460619|0.012873312423291967|
|        [82581]|   [82580]| 0.760989010989011| 30.09888035373472|0.015109777717168963|
|        [22746]|   [22748]|0.8869565217391304|52.792631281761

<div style="page-break-before: always;"></div>


## Q9. Final Association Rules [8 pts]

Present the resulting Association Rules <u>in terms of the original StockCodes and Descriptions</u>, in descending order of lift.

In [127]:
df_association_rules = model_retail.associationRules.toPandas()
# sorting by lift
df_association_rules_sorted = df_association_rules.sort_values(by='lift', ascending=False).reset_index()
# drop byproduct index column
df_association_rules_sorted = df_association_rules_sorted.drop(df_association_rules_sorted.columns[0], axis=1)
# creating new column to store Description
df_association_rules_sorted.insert(1, 'A-Description', df_association_rules_sorted['antecedent'].copy())
df_association_rules_sorted.insert(3, 'C-Description', df_association_rules_sorted['consequent'].copy())
# print(df_association_rules_sorted)
# replacing StockCodes with Description
description_dict = df_clean.set_index('StockCode')['Description'].to_dict()
def convert_to_description(s):
  if len(s) == 1:
    return description_dict[s[0]]
  else:
    return [description_dict[i] for i in s]
df_association_rules_sorted['A-Description'] = df_association_rules_sorted['A-Description'].map(convert_to_description)
df_association_rules_sorted['C-Description'] = df_association_rules_sorted['C-Description'].map(convert_to_description)
df_association_rules_sorted
#

/content/spark-3.2.1-bin-hadoop3.2/python/pyspark/sql/context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


,antecedent,A-Description,consequent,C-Description,confidence,lift,support
0,[22746],POPPY'S PLAYHOUSE LIVINGROOM,[22745],POPPY'S PLAYHOUSE BEDROOM,0.830435,54.762394,0.010419
1,[22746],POPPY'S PLAYHOUSE LIVINGROOM,[22748],POPPY'S PLAYHOUSE KITCHEN,0.886957,52.792631,0.011128
2,[22748],POPPY'S PLAYHOUSE KITCHEN,[22745],POPPY'S PLAYHOUSE BEDROOM,0.766234,50.528707,0.012873
3,[22745],POPPY'S PLAYHOUSE BEDROOM,[22748],POPPY'S PLAYHOUSE KITCHEN,0.848921,50.528707,0.012873
4,[21899],GARAGE KEY FOB,[21900],"KEY FOB , SHED",0.782027,43.842525,0.011155
5,[21086],SET/6 RED SPOTTY PAPER CUPS,[21094],SET/6 RED SPOTTY PAPER PLATES,0.787440,41.721782,0.013337
6,[21124],SET/10 BLUE POLKADOT PARTY CANDLES,[21122],SET/10 PINK POLKADOT PARTY CANDLES,0.801336,39.757738,0.013092
7,"[22698, 22699]","[PINK REGENCY TEACUP AND SAUCER, ROSES REGENCY...",[22697],GREEN REGENCY TEACUP AND SAUCER,0.895868,34.721979,0.014782
8,"[22698, 22423]","[PINK REGENCY TEACUP AND SAUCER, REGENCY CAKES...",[22697],GREEN REGENCY TEACUP AND SAUCER,0.890951,34.531426,0.010473
9,[22698],PINK REGENCY TEACUP AND SAUCER,[22697],GREEN REGENCY TEACUP AND SAUCER,0.841060,32.597728,0.017319


https://colab.research.google.com/drive/1AfxGRNd8NodZ4OiqR3KKLDzzigshHmSB?usp=sharing